In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/content/drive/MyDrive/market_data_train.csv')

In [3]:
#df['symbol'].unique()

In [4]:
#df['time'].unique()

In [5]:
df.head(5)

,symbol,time,open,high,low,close,market_cap,n_trades,volume,taker_buy_volume
0,1INCH,2020-12-25 06:00:00,0.2000,3.0885,0.2000,2.5826,6.707592e+07,48768,3.553052e+07,1.724763e+07
1,1INCH,2020-12-25 07:00:00,2.5824,2.6900,2.2249,2.5059,6.508385e+07,31099,2.244087e+07,9.861481e+06
2,1INCH,2020-12-25 08:00:00,2.5152,2.8870,2.3609,2.6237,6.814338e+07,33001,2.130043e+07,1.160785e+07
3,1INCH,2020-12-25 09:00:00,2.6318,2.8247,2.4650,2.6134,6.787587e+07,30459,1.749181e+07,9.087566e+06
4,1INCH,2020-12-25 10:00:00,2.6104,2.7498,2.5629,2.6365,6.847583e+07,21023,9.919400e+06,4.798067e+06


In [6]:
df.isna().sum()

symbol              0
time                0
open                0
high                0
low                 0
close               0
market_cap          0
n_trades            0
volume              0
taker_buy_volume    0
dtype: int64

In [7]:
df = df.set_index(['time','symbol'])

In [8]:
df['x1']=df.open/df.close-1 #  daily return

In [9]:
df['x2'] = df.volume.apply(np.log) 

In [10]:
df['x3'] = df.groupby(level='symbol').volume.diff(3) # change since prior 3 hour
df['x4'] = df.groupby(level='symbol').volume.diff(12) # change since prior 12 hour
df['x5'] = df.groupby(level='symbol').volume.diff(24) # change since prior 24 hour

In [11]:
# close price 3 hour moving average
ma_3 = lambda x: x.rolling(3).mean()
df['x6'] = df.close.groupby(level='symbol').apply(ma_3)
ma_12 = lambda x: x.rolling(12).mean()
df['x7'] = df.close.groupby(level='symbol').apply(ma_12)
ma_24 = lambda x: x.rolling(24).mean()
df['x8'] = df.close.groupby(level='symbol').apply(ma_24)

In [12]:
# momentum 1,3,12,24 H
df['x9']=(df.groupby(level='symbol').close.shift(0)/df.groupby(level='symbol').close.shift(1))
df['x10']=(df.groupby(level='symbol').close.shift(0)/df.groupby(level='symbol').close.shift(3))
df['x11']=(df.groupby(level='symbol').close.shift(0)/df.groupby(level='symbol').close.shift(12))
df['x12']=(df.groupby(level='symbol').close.shift(0)/df.groupby(level='symbol').close.shift(24))

In [13]:
# rank 1h market cap (pct)
df['x13'] = df.market_cap.groupby(level='time').rank(pct=True) 

In [14]:
stds = df.groupby(level='symbol')['x9'].std()
df.reset_index(level='symbol',inplace=True)
symbols = df['symbol'].unique()

In [15]:
stds

symbol
1INCH    0.024800
AAVE     0.020458
ACM      0.030794
ADA      0.012918
AION     0.020231
           ...   
YFII     0.021821
ZEC      0.013693
ZEN      0.020482
ZIL      0.015728
ZRX      0.015140
Name: x9, Length: 219, dtype: float64

In [16]:
# symbols

In [17]:
# стандартные отклонения
for i in range(len(symbols)): 
  #print(symbols[i],stds[i])
  df.loc[df['symbol']==symbols[i] , ['x14']]=stds[i] 

In [18]:
df.reset_index(level='time',inplace=True)

In [19]:
df.set_index(['time','symbol'],inplace=True)

In [20]:
df.sort_index(inplace=True)
df.tail(24)

open          high  ...       x13       x14
time                symbol                              ...                    
2021-05-31 20:00:00 UNI        26.908000     27.480000  ...  0.968037  0.021423
                    UTK         0.314300      0.319000  ...  0.347032  0.024623
                    VET         0.117310      0.120070  ...  0.926941  0.015735
                    VITE        0.088980      0.090360  ...  0.105023  0.021480
                    VTHO        0.009213      0.009270  ...  0.561644  0.024623
                    WAN         0.900200      0.911600  ...  0.374429  0.016963
                    WAVES      12.880000     12.978000  ...  0.771689  0.014971
                    WIN         0.000513      0.000519  ...  0.383562  0.017150
                    WING       20.219000     20.583000  ...  0.077626  0.028232
                    WNXM       70.289000     70.499000  ...  0.310502  0.021401
                    WRX         1.958000      2.358200  ...  0.639269  0.022479
                    WTC         0.826400      0.843900  ...  0.182648  0.018581
                    XEM         0.188400      0.189000  ...  0.799087  0.021300
                    XLM         0.389860      0.392590  ...  0.940639  0.013295
                    XMR       258.380000    261.130000  ...  0.899543  0.011447
                    XRP         0.986200      0.996400  ...  0.981735  0.013008
                    XTZ         3.463700      3.497000  ...  0.876712  0.015578
                    XVS        31.956000     33.199000  ...  0.566210  0.031782
                    YFI     45323.690000  46273.960000  ...  0.794521  0.022131
                    YFII     1855.180000   1869.500000  ...  0.205479  0.021821
                    ZEC       153.250000    156.270000  ...  0.812785  0.013693
                    ZEN       105.000000    106.320000  ...  0.744292  0.020482
                    ZIL         0.110580      0.111830  ...  0.748858  0.015728
                    ZRX         0.988200      0.999800  ...  0.684932  0.015140

[24 rows x 22 columns]

In [21]:
df['i1']=df.groupby('time')['market_cap'].transform(lambda x: x/x.sum()) # индикатор взвешенный по капитализаций за 1 час

In [22]:
# MA для i1
df['i2'] = df.i1.groupby(level='symbol').apply(ma_3)
df['i3'] = df.i1.groupby(level='symbol').apply(ma_12)
df['i4'] = df.i1.groupby(level='symbol').apply(ma_24)

In [23]:
# STD для i1
df['i5']=df.groupby(level='symbol')['i1'].transform(lambda x: x.std())

In [24]:
# будущая доходность
df['y1']= df.groupby(level='symbol').close.pct_change(3)
df['y2']= df.groupby(level='symbol').close.pct_change(12)
df['y3']= df.groupby(level='symbol').close.pct_change(24)

In [25]:
df.reset_index(['time','symbol'],inplace=True)

In [26]:
df.sort_values(by=['symbol','time'],inplace=True)
df=df.reset_index(drop=True)
df=df.drop(columns=df.columns[2:10])

In [27]:
df.head(5)

,time,symbol,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,i1,i2,i3,i4,i5,y1,y2,y3
0,2020-12-25 06:00:00,1INCH,-0.922559,17.385903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.514451,0.0248,0.000113,NaN,NaN,NaN,0.000138,NaN,NaN,NaN
1,2020-12-25 07:00:00,1INCH,0.030528,16.926395,NaN,NaN,NaN,NaN,NaN,NaN,0.970301,NaN,NaN,NaN,0.491329,0.0248,0.000109,NaN,NaN,NaN,0.000138,NaN,NaN,NaN
2,2020-12-25 08:00:00,1INCH,-0.041354,16.874238,NaN,NaN,NaN,2.570733,NaN,NaN,1.047009,NaN,NaN,NaN,0.520231,0.0248,0.000114,0.000112,NaN,NaN,0.000138,NaN,NaN,NaN
3,2020-12-25 09:00:00,1INCH,0.007041,16.677243,-1.803870e+07,NaN,NaN,2.581000,NaN,NaN,0.996074,1.011926,NaN,NaN,0.520231,0.0248,0.000113,0.000112,NaN,NaN,0.000138,0.011926,NaN,NaN
4,2020-12-25 10:00:00,1INCH,-0.009899,16.110003,-1.252147e+07,NaN,NaN,2.624533,NaN,NaN,1.008839,1.052117,NaN,NaN,0.520231,0.0248,0.000113,0.000113,NaN,NaN,0.000138,0.052117,NaN,NaN
